##### 数据构成
总共 100,000 条评论，其中包括：

- 25,000 条标注的训练集（train/）

- 25,000 条标注的测试集（test/）

- 50,000 条无标注的评论（train/unsup/）

##### 标签定义

评分 ≤ 4 的评论标为 负面（neg），评分 ≥ 7 的评论标为 正面（pos）。中性评分（5-6 分）被排除在标注数据之外

##### 文件结构

train/pos/, train/neg/, test/pos/, test/neg/, train/unsup/ 五个文件夹，每条评论是一个 .txt 文件，命名格式为 [id]_[rating].txt

urls_*.txt：每条评论对应的 IMDb 页面链接（按行号对应）

*.feat：已经生成的词袋特征（BoW），采用 LIBSVM 格式，配合 imdb.vocab 可映射词索引

imdb.vocab：词汇表

imdbEr.txt：每个词的期望评分值，反映其平均情感极性

##### 数据预处理原则
每部电影最多只收录 30 条评论，以避免训练集中出现过多与某部电影相关的偏差

训练集与测试集中的电影完全不重叠

In [3]:
import os
import torch
import random
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

# 设置随机种子
torch.manual_seed(42)

# 路径设置
DATA_DIR = './深度学习/data_循环'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')

# 自定义读取数据
def load_imdb_data(directory):
    data = []
    for label in ['pos', 'neg']:
        path = os.path.join(directory, label)
        for fname in os.listdir(path):
            if fname.endswith('.txt'):
                with open(os.path.join(path, fname), 'r', encoding='utf-8') as f:
                    text = f.read()
                    data.append((text, 1 if label == 'pos' else 0))
    return data

train_data = load_imdb_data(TRAIN_DIR)
test_data = load_imdb_data(TEST_DIR)
random.shuffle(train_data)
random.shuffle(test_data)
print(train_data.head())

OSError: [WinError 127] 找不到指定的程序。